### neo4j 取数据解析

In [1]:
from flask import Flask, jsonify, render_template
from py2neo import Graph

#app = Flask(__name__)
graph = Graph(
     host = "localhost", # neo4j 搭载服务器的ip地址，ifconfig可获取到
    http_port = 7474, # neo4j 服务器监听的端口号
    user = "neo4j", # 数据库user name，如果没有更改过，应该是neo4j
    password = "qiyu" # 自己设定的密码
)

def build_node(index,n):
    d = dict(zip(n['n'].keys(),n['n'].values()))
    d.update({"id":str(n['n'].identity)})
    d.update({"label":str(n['n'].labels)[1:]})
    return {"data": d}

def build_edge(r):
    d={
    "source": r['r'].start_node.identity,
    "target": r['r'].end_node.identity,
    "relationship":list(r['r'].relationships[0].types())[0]}
    return {"data":d}

In [2]:
#@app.route('/')
def index():
    return render_template('index.html')

#@app.route('/graph')
def get_graph():
    nodes = graph.run('MATCH (n) RETURN n').data()
    edges = graph.run('MATCH ()-[r]->() RETURN r').data()

    return jsonify(elements = {"nodes": [build_node(index,n) for index, n in enumerate(nodes)],
                                 "edges": [build_edge(r) for r in edges]})

In [3]:
nodes = graph.run('MATCH (n) RETURN n').data()
edges = graph.run('MATCH ()-[r]->() RETURN r').data()
data = {"nodes": [build_node(index,n) for index, n in enumerate(nodes)],
                                 "edges": [build_edge(r) for r in edges]}

data

{'edges': [],
 'nodes': [{'data': {'Link': 'https://baike.baidu.com/item/Logistic%E6%A8%A1%E5%9E%8B/355091?fromtitle=Logistic%20regression&fromid=18080373&fr=aladdin',
    'ModelDescription': 'Logistic Regression',
    'ModelName': 'LR',
    'ModelType': 'Linear',
    'id': '0',
    'label': 'Model'}},
  {'data': {'Link': 'https://tech.meituan.com/deep_understanding_of_ffm_principles_and_practices.html',
    'ModelDescription': 'Factorization Machine',
    'ModelName': 'FM',
    'ModelType': 'Nonlinear',
    'id': '1',
    'label': 'Model'}},
  {'data': {'Link': 'https://baike.baidu.com/item/LDA/13489644',
    'ModelDescription': 'Latent Dirichlet Allocation',
    'ModelName': 'LDA',
    'ModelType': 'Graphic',
    'id': '2',
    'label': 'Model'}},
  {'data': {'Link': 'https://baike.baidu.com/item/Word2vec/22660840?fr=aladdin',
    'ModelDescription': 'Word to Vector',
    'ModelName': 'Word2Vec',
    'ModelType': 'Neural',
    'id': '3',
    'label': 'Model'}},
  {'data': {'Link': 'h

In [6]:
node_indices = [e['data']['id'] for e in data['nodes']]
node_indices

['0', '1', '2', '3', '4']

In [7]:
edge_starts =  [e['data']['source'] for e in data['edges']]
edge_end =  [e['data']['target'] for e in data['edges']]

### pyecharts<br>
参考https://pyecharts.github.io/pyecharts.js-app/#https://raw.githubusercontent.com/pyecharts/pyecharts-app/master/app/charts/graph.py

In [18]:
from pyecharts import Graph, Page, Style

ERROR:lml.utils:failed to import pyecharts_snapshot
Traceback (most recent call last):
  File "/Users/a123/anaconda3/lib/python3.6/site-packages/lml/utils.py", line 43, in do_import
    plugin_module = __import__(plugin_module_name)
ModuleNotFoundError: No module named 'pyecharts_snapshot'


In [19]:
def create_charts():
    page = Page()

    style = Style(
        width=1100, height=600
    )

    nodes = [{"name": "结点1", "symbolSize": 10},
             {"name": "结点2", "symbolSize": 20},
             {"name": "结点3", "symbolSize": 30},
             {"name": "结点4", "symbolSize": 40},
             {"name": "结点5", "symbolSize": 50},
             {"name": "结点6", "symbolSize": 40},
             {"name": "结点7", "symbolSize": 30},
             {"name": "结点8", "symbolSize": 20}]
    links = []
    for i in nodes:
        for j in nodes:
            links.append({"source": i.get('name'), "target": j.get('name')})
    chart = Graph("关系图-力引导布局", **style.init_style)
    chart.add("", nodes, links, graph_repulsion=8000, line_color='#aaa')
    page.add(chart)

#     chart = Graph("关系图-环形布局", **style.init_style)
#     chart.add("", nodes, links, is_label_show=True, graph_repulsion=8000,
#               graph_layout='circular', label_text_color=None)
#     page.add(chart)

#     nodes, links, categories, cont, mid, _ = WEIBO
#     chart = Graph("关系图-微博转发", **style.init_style)
#     chart.add("", nodes, links, categories, label_pos="right", graph_repulsion=50,
#               is_legend_show=False, line_curve=0.2, label_text_color=None)
#     page.add(chart)

    return page
create_charts().render()

### bokeh

In [12]:
from bokeh.plotting import figure,output_file,show
from bokeh.models import GraphRenderer,StaticLayoutProvider, Oval
from bokeh.palettes import Spectral8

In [13]:
graph = GraphRenderer()
graph.node_renderer.data_source.add(node_indices, 'index')
graph.node_renderer.data_source.add(Spectral8[:2], 'color')
graph.node_renderer.glyph = Oval(height=0.1, width=0.1)
graph.node_renderer.data_source.data = dict(
    index=node_indices,
    fill_color=Spectral8[:2])

graph.edge_renderer.data_source.data = dict(
    start=edge_starts,
    end=edge_end)

/Users/a123/anaconda3/lib/python3.6/site-packages/bokeh/models/sources.py:137: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('color', 2), ('index', 5)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/Users/a123/anaconda3/lib/python3.6/site-packages/bokeh/models/sources.py:137: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('fill_color', 2), ('index', 5)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))


In [14]:
import math
circ = [int(i)*2*math.pi/8 for i in node_indices]
x = [math.cos(i) for i in circ]
y = [math.sin(i) for i in circ]

graph_layout = dict(zip(node_indices, zip(x, y)))
graph.layout_provider = StaticLayoutProvider(graph_layout=graph_layout)


plot = figure(title='Graph Layout Demonstration', x_range=(-1.1,1.1), y_range=(-1.1,1.1),
              tools='', toolbar_location=None)
plot.renderers.append(graph)

output_file('graph.html')
show(plot)

In [37]:
circ,x,y

([0.0, 0.7853981633974483],
 [1.0, 0.7071067811865476],
 [0.0, 0.7071067811865475])

### 生成网页+链接

In [13]:
from bokeh.models import ColumnDataSource, OpenURL, TapTool
from bokeh.plotting import figure, output_file, show

output_file("openurl.html")

p = figure(plot_width=400, plot_height=400,
           tools="tap", title="Click the Dots")

source = ColumnDataSource(data=dict(
    x=[1, 2, 3, 4, 5],
    y=[2, 5, 8, 2, 7],
    color=["navy", "orange", "olive", "firebrick", "gold"]
    ))

p.circle('x', 'y', color='color', size=20, source=source)

url = "http://www.colors.commutercreative.com/@color/"
taptool = p.select(type=TapTool)
taptool.callback = OpenURL(url=url)

show(p)